In [1]:
import numpy as np
import cv2
from PIL import Image

from Algorithm import key_expansion, ciphering_block, key_update, deciphering_block, s, Arnold
from Image import blockshaped, From_blocks_to_arr_image
from Metrics import Entr, Hist, EQ

In [2]:
# шифрование картинки

k1 = [1,32,3,234,15,156,7,8,9,40,1,2,3,43,5,64]
k2 = [1,2,35,54,55,26,73,8,9,50,21,2,3,45,5,65]
k3 = [1,27,3,24,25,6,7,8,9,0,13,2,23,4,5,64]
k4 = [143,25,33,4,5,6,73,8,39,0,1,2,3,4,5,6]

img_path = 'station.jpg'
img = cv2.imread(img_path, 0)

imgb = blockshaped(img, 16, 16)
Nb = len(imgb)
imgb_c = [0] * Nb
imgb_d = [0] * Nb

key_B = key_expansion(k1, k2, k3, k4)

for i in range(Nb):
    B = imgb[i]
    B_C = ciphering_block(B, key_B, 8)
    imgb_c[i] = B_C
    key_B = key_update(key_B, B_C, B, k1, k2, k3, k4, 8)

In [3]:
k = int(len(img[0]) / 16)

arr2 = From_blocks_to_arr_image(imgb_c,Nb,k)

im_1 = Image.fromarray(arr2, 'L')
im_1.save("encrypted.jpg")

In [4]:
# дешифрование зашифрованной картинки
key_B = key_expansion(k1, k2, k3, k4)

for i in range(Nb):
    B_C = imgb_c[i]
    B_D = deciphering_block(B_C, key_B, 8)
    imgb_d[i] = B_D
    key_B = key_update(key_B, B_C, B_D, k1, k2, k3, k4, 8)

In [5]:
arr2 = From_blocks_to_arr_image(imgb_d,Nb,k)

im_1 = Image.fromarray(arr2, 'L')
im_1.save("decrypted.jpg")

In [6]:
#Получаем key_B из первого блока зашифрованной и расшифрованной картинок
key_B_0_hk = deciphering_block(imgb_c[0],imgb_d[0],8)
print(np.all(key_B_0_hk == key_expansion(k1, k2, k3, k4)))

True


/Users/apple/Securing_DICOM_Images/Algorithm.py:19: RuntimeWarning: overflow encountered in ubyte_scalars
  p_i = (c_i - k_i) % (2**n)


In [7]:
def Arn_hk(mat):
    mat_A = Arnold(mat)
    mat_A_1 = mat_A
    while(np.any(mat_A != mat)):
        mat_A_1 = mat_A
        mat_A = Arnold(mat_A)
    return mat_A_1

In [8]:
key_B_2 = deciphering_block(imgb_c[1],imgb_d[1],8)

s_1 = (Arn_hk(key_B_2) - np.outer(imgb_c[0].sum(0),imgb_d[0].sum(1)) - key_B_0_hk) % (2**8)

/Users/apple/Securing_DICOM_Images/Algorithm.py:19: RuntimeWarning: overflow encountered in ubyte_scalars
  p_i = (c_i - k_i) % (2**n)


In [9]:
print(np.all(s_1 == s(k1, k2, k3, k4)%(2**8)))#Полученая нами матрица совпадает с 

True


In [10]:
s_1[7][8]

16.0

In [11]:
k1_k2_k3_k4_hk = np.zeros((16,4))

In [12]:
k1_k2_k3_k4_hk

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [13]:
def get_kv_vich(n):
    vich_arr = []
    for t in range(0,2**n):
        vich_arr.append(t**2 % (2**n))
    return list(dict.fromkeys(vich_arr))

In [14]:
get_kv_vich(8)

[0,
 1,
 4,
 9,
 16,
 25,
 36,
 49,
 64,
 81,
 100,
 121,
 144,
 169,
 196,
 225,
 33,
 68,
 105,
 185,
 228,
 17,
 113,
 164,
 217,
 73,
 132,
 193,
 65,
 201,
 89,
 241,
 145,
 57,
 233,
 161,
 97,
 41,
 249,
 209,
 177,
 153,
 137,
 129]

In [15]:
from itertools import combinations_with_replacement 
Kv_vich = get_kv_vich(8)
i_j_diag_pret = []
for i in range(16):
    for j in range(16):
        if i==j:
            pret_list = []
            for p in combinations_with_replacement(Kv_vich,4):
                p_1 = np.array(p)
                if p_1.sum()%(2**8) == s_1[i][j]:
                    pret_list.append(p)
            i_j_diag_pret.append(pret_list)

In [16]:
for i in i_j_diag_pret:
    print(len(i))

1651
512
1662
20
1662
1662
1651
2
1651
1651
1651
45
1662
746
1651
200


In [17]:
i_j_diag_pret[7]

[(0, 0, 0, 0), (64, 64, 64, 64)]

In [18]:
op = []
for j in get_kv_vich(8):
    o=0
    for i in range(256):
        if i*i % 256 == j:
            o+=1
    op.append(o)
print(op)


[16, 4, 8, 4, 16, 4, 8, 4, 16, 4, 8, 4, 16, 4, 8, 4, 4, 8, 4, 4, 8, 4, 4, 8, 4, 4, 8, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]


In [19]:
len(op)

44

In [20]:
print(len(i_j_diag_pret))

16


In [21]:
125*125 % 256

9